## function calling


## 진짜 실습

In [ ]:
# 운영체제(OS) 관련 기능을 사용하기 위한 모듈 불러오기
# → 환경변수(Environment Variable)를 읽기 위해 필요
import os

# .env 파일을 읽어서 환경변수로 등록해주는 함수 불러오기
from dotenv import load_dotenv

# 현재 프로젝트 폴더의 .env 파일을 읽어서
# 안에 있는 변수들을 시스템 환경변수로 로드(등록)
load_dotenv()

# 환경변수 중에서 "OPENAI_API_KEY" 라는 이름의 값을 가져와서
# OPENAI_API_KEY 파이썬 변수에 저장
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# JSON 형식으로 데이터를 변환하기 위한 파이썬 내장 라이브러리
# 딕셔너리 → JSON 문자열 형태로 바꿀 때 사용
import json

# OpenAI API 서버와 통신하기 위한 OpenAI 클래스 불러오기
# (이 코드에서는 아직 사용하지 않았지만, Function Calling과 연동할 때 필요)
from openai import OpenAI




# 현재 날씨 정보를 반환하는 함수 정의
# location : 지역 이름 (예: "Seoul")
# unit : 온도 단위 (기본값은 'celsius' = 섭씨)
def get_current_weather(location,unit='celsius'):

    # 날씨 정보를 저장할 딕셔너리(데이터 묶음) 생성
    # 실제 날씨 API가 아니라 테스트용 가짜 데이터(mock 데이터)
    weather_info={
        # 함수로 전달받은 지역 이름을 그대로 저장
        "location":location,
        # 현재 온도 (문자열 형태)
        # 실제 계산값이 아니라 항상 "25"로 고정된 테스트 값
        "temperature":"25",
         # 온도 단위
        # 현재는 항상 섭씨(celsius)로 고정
        "unit":"celsius",
        # 날씨 예보 목록 (리스트 자료형)
        # sunny = 맑음
        # windy = 바람 많음
        "forecast":["sunny","windy"],
    }

    # weather_info 딕셔너리를 JSON 문자열 형태로 변환해서 반환
    # OpenAI function calling 또는 API 통신에서 사용하기 위함
    return json.dumps(weather_info)

In [ ]:
# OpenAI Function Calling(툴 호출)에서 사용할 함수 목록을 정의
# GPT에게 "사용 가능한 함수 메뉴판"을 알려주는 역할
functions=[
     # 하나의 함수 정보를 딕셔너리 형태로 작성
    {   
        # GPT가 호출할 함수의 이름
        # 실제 파이썬 함수 이름과 반드시 같아야 함
        # 예: def get_current_weather(...)
        "name":"get_current_weather",
        # 이 함수가 어떤 역할을 하는지 설명
        # GPT가 여러 함수 중 어떤 것을 선택할지 판단할 때 참고함
        "description":"Get the current weather in a given location",

        # 함수에 전달될 입력값(파라미터)들의 구조 정의
        "parameters":{
            # 입력값 전체 구조 타입
            # object = JSON 객체 = 파이썬 딕셔너리 형태
            "type":"object",
            # 실제로 받을 수 있는 파라미터 목록
            "properties":{
                # location 파라미터 정의
                "location":{
                    # location은 문자열(string) 타입만 허용
                    # 예: "Seoul", "New York
                    "type":'string',
                    # GPT에게 보여주는 설명 문구
                    # 어떤 형식으로 입력해야 하는지 안내
                    "description":"The city and state,e.g Seoul",
                },
                 # unit 파라미터 정의 (온도 단위)
                  # 문자열 타입만 허용
                   # enum = 선택 가능한 값 제한
                    # 반드시 아래 둘 중 하나만 가능
                    # "celsius" 또는 "fahrenheit"
                "unit":{"type":"string","enum":["celsius","fahrenheit"]},
            },
             # 필수 입력값 지정
            # location은 반드시 필요함
            # unit은 필수가 아니므로 없어도 됨
            "required":["location"],
        },
    }
]

In [ ]:
# GPT에게 보낼 대화 내용을 저장하는 리스트
# role = 누가 말했는지 (user = 사용자)
# content = 실제 질문 내용
messages = [{"role": "user",  # 사용자가 보낸 메시지라는 의미
             "content": "What's the weather like in Seoul?"}]  # GPT에게 묻는 질문

# OpenAI API 서버와 통신하기 위한 클라이언트(연결 객체) 생성
# 내부적으로 API KEY를 불러와 인증하고 서버 연결 준비를 함
client = OpenAI()

# GPT 모델에게 채팅 요청을 보내는 부분
# response 변수에 GPT 서버의 응답 결과가 저장됨
response=client.chat.completions.create(
    # 사용할 AI 모델 지정
    # gpt-4.1-mini : 빠르고 비용이 저렴하며 Function Calling 지원
    model="gpt-4.1-mini",
    # GPT에게 전달할 대화 내용
    # 사용자가 "서울 날씨 알려줘" 라고 질문한 정보가 들어 있음
    messages=messages,
    # GPT가 사용할 수 있는 함수 목록 전달
    # 이 정보 덕분에 GPT가 "아 날씨 함수 써야겠다" 라고 판단 가능
    functions=functions

)

# GPT가 반환한 응답 객체를 JSON 형태로 보기 좋게 출력
# indent=2 : 들여쓰기 2칸으로 정렬해서 사람이 읽기 쉽게 출력
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CyTNotZV1K8JL4qiu0NxBf2eiuIms",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": {
          "arguments": "{\"location\":\"Seoul\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1768528664,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 74,
    "total_tokens": 95,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tok

In [38]:
# GPT 응답 객체(response) 안에서
# 실제 메시지 부분만 꺼내서 response_message 변수에 저장
# 이 안에는 function_call 정보가 들어 있음
response_message=response.choices[0].message

# 전체 GPT 응답 내용을 그대로 출력
# 디버깅용: GPT가 어떤 데이터를 보냈는지 구조 확인할 때 사용
print(response)

# GPT가 호출할 수 있는 함수 목록(연결 테이블) 생성
# 문자열 함수 이름 → 실제 파이썬 함수 객체 연결
available_functions={
     # GPT가 보내는 name 값과 반드시 동일해야 함
    "get_current_weather":get_current_weather,
}

# GPT가 "이 함수를 호출해라" 라고 지정한 함수 이름 가져오기
# 예: "get_current_weather"
function_name=response_message.function_call.name

# 함수 이름을 이용해서 실제 실행할 파이썬 함수 찾기
# 딕셔너리에서 이름에 해당하는 함수 객체를 꺼냄
function_to_call=available_functions[function_name]

# GPT가 보내준 함수 인자(arguments)는 JSON 문자열 형태
# json.loads()를 사용해서 문자열 → 파이썬 딕셔너리로 변환
# 예: '{"location":"Seoul","unit":"celsius"}'
# → {"location":"Seoul","unit":"celsius"}
function_args=json.loads(response_message.function_call.arguments)


# 실제 파이썬 함수 실행
# GPT가 보내준 인자값을 함수에 전달
# get_current_weather("Seoul", "celsius") 와 동일한 동작
function_response=function_to_call(
    # 지역 정보 전달
    location=function_args.get("location"),
    # 온도 단위 전달 (없으면 None)
    unit=function_args.get("unit"),
)

# 실행 결과 출력
# get_current_weather 함수가 반환한 JSON 문자열 출력
print(function_response)

ChatCompletion(id='chatcmpl-CyTNotZV1K8JL4qiu0NxBf2eiuIms', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Seoul","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1768528664, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_376a7ccef1', usage=CompletionUsage(completion_tokens=21, prompt_tokens=74, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
{"location": "Seoul", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [ ]:
# GPT가 방금 반환한 메시지(response_message)를
# 기존 대화 기록(messages)에 추가
# 이 메시지 안에는 "어떤 함수를 호출하라"는 function_call 정보가 들어 있음
# 이렇게 추가해야 GPT가 자신의 이전 응답을 대화 맥락으로 기억할 수 있음
messages.append(response_message)


# 함수 실행 결과를 GPT에게 다시 전달하기 위한 메시지 추가
# role을 "function"으로 설정하면
# GPT는 이것을 "함수 실행 결과"로 인식함
messages.append(
    {    # 이 메시지는 사람이 말한 것도, GPT가 말한 것도 아닌
        # 실제 함수 실행 결과라는 의미
        "role":"function",
        # 어떤 함수의 결과인지 GPT에게 알려주는 부분
        # 반드시 GPT가 요청한 함수 이름과 같아야 함
        # 예: "get_current_weather"
        "name":function_name,

         # 실제 함수 실행 결과 데이터
        # 보통 JSON 문자열 형태
        # GPT가 이 값을 읽어서 최종 자연어 답변을 생성함
        "content":function_response,
    }
)

In [34]:
print(messages)

[{'role': 'user', 'content': "What's the weather like in Seoul?"}, ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Seoul","unit":"celsius"}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Seoul", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}'}]


In [ ]:
# 함수 실행 결과까지 모두 포함된 messages(대화 기록)를 가지고
# GPT에게 다시 요청을 보내는 부분
# 이 단계에서 GPT는 "최종 사람 말 답변"을 생성함
second_response=client.chat.completions.create(
    # 사용할 AI 모델 지정
    # gpt-4.1-mini : 빠르고 비용이 저렴하며 Function Calling 지원
    model='gpt-4.1-mini',

    # GPT에게 전달할 전체 대화 기록
    # 여기에는 다음 정보가 모두 들어 있음:
    # 1) 사용자 질문
    # 2) GPT가 요청한 함수 호출 정보
    # 3) 실제 함수 실행 결과
    # GPT는 이 내용을 보고 최종 답변을 생성함
    messages=messages,
)
# GPT가 생성한 최종 응답 전체를 JSON 형태로 출력
# indent=2 : 들여쓰기 2칸으로 정렬해서 사람이 보기 쉽게 출력
print(second_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-CyTNpjiCLhfRStUGVX0j0N2OmYuXU",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The weather in Seoul is currently sunny and windy, with a temperature of 25°C.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1768528665,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_376a7ccef1",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 76,
    "total_tokens": 94,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
